# 6 Redirection

I/O redirection
- input / output redirection

|Command| Action|
|-|-|
|cat| Concatenate files|
|sort| Sort lines of text|
|uniq| Report or omit repeated lines|
|grep| Print lines matching a pattern|
|wc| Print newline, word, and byte counts for each file|
|head| Output the first part of a file|
|tail| Output the last part of a file|
|tee| Read from standard input and write to standard output and files|

## Standard input, Output, and Error

Program outputs generally fall into two categories
- program results
- status and errors

Programs actually send their results to a special file called `standard output` and their status messages to `standard error`, linked to the screen not saved to disk file by default. 

Many programs take input from a facility called `standard input`, which is by default attached to the keyboard..

I/O redirection allows us to changed where output goes and where input comes from

## Redirecting Standard Output

To redirect standard output use
- output `>` filename 

In [3]:
!ls -l /usr/bin > ls-output.txt

In [4]:
!ls -l ls-output.txt

-rwxrwxrwx 1 yemane yemane 81057 Nov 29 19:50 ls-output.txt


Try with a fake directory

In [5]:
!ls -l /bin/usr > ls-output.txt

ls: cannot access '/bin/usr': No such file or directory


Now the file is zero length. The destination file is always rewritten from the beginning. Since the command had an error, which is sent to stderror (stdoutput), nothing is written to file.

In [6]:
!ls -l ls-output.txt

-rwxrwxrwx 1 yemane yemane 0 Nov 29 19:53 ls-output.txt


Truncate existing file or create an empty file

In [9]:
!> ls-output.txt

To append redirection output use
- output `>>` filename 

In [10]:
!ls -l /usr/bin >> ls-output.txt
!ls -l /usr/bin >> ls-output.txt
!ls -l /usr/bin >> ls-output.txt

We repeated the command three times, resulting in an output file 
three times as large

In [11]:
!ls -l ls-output.txt

-rwxrwxrwx 1 yemane yemane 243171 Nov 29 19:59 ls-output.txt


## Redirecting Standard Error

To redirect standard error, we must refer to its file descriptor. While we have referred to the first three of these file streams as standard input, output, and error, the shell references them internally as file descriptors 0, 1, and 2, respectively. The shell provides a notation for redirecting files using the file descriptor number. Because standard error is the same as file descriptor number 2, we can redirect standard error with this notation:

The file descriptor 2 is placed immediately before the redirection operator

In [12]:
!ls -l /bin/usr 2> ls-error.txt

### Redirecting Standard Output and Standard Error to One File

To do this, we must redirect both the standard output and standard error at the same time.

In [14]:
!ls -l /bin/usr &> ls-output.txt

### Disposing of Unwanted Output

The system provides a way to do this by redirecting output to a 
special file called `/dev/null`. This file is a system device often referred to as a bit bucket, which accepts input and does nothing with it. To suppress error messages from a command.

In [15]:
!ls -l /bin/usr 2> /dev/null

## Redirecting Standard Input

#### cat: Concatenate Files
- Multiple files name as: 
    - movie.mpeg.001 movie.mpeg.002 ... movie.mpeg.099
- Join them by
    - cat movie.mpeg.0* > movie.mpeg
    
Because wildcards always expand in sorted order, the arguments will be 
arranged in the correct order.

In the absence of filename arguments, cat copies standard input to standard output, so we see our line of text repeated. We can use this behavior to create short text files. Let’s say we wanted to create a file called lazy_dog.txt containing the text in our example

```sh
$ cat > lazy_dog.txt
$ The quick brown fox jumped over the lazy dog.
$ ctrl-D
```

Using the `<` redirection operator, we change the source of standard input 
from the keyboard to the file lazy_dog.txt

In [1]:
!cat < lazy_dog.txt

The quick brown fox jumped over the lazy dog.


## Pipelines

The capability of commands to read data from standard input and send to 
standard output is utilized by a shell feature called pipelines. Using the pipe operator `|`, the standard output of one command can be piped into the standard input of another.

```
command1 | command2
```

### Filters

It is possible to put several commands together into a pipeline. Frequently, the commands used this way are referred to as `filters`. Filters take input, change it somehow, and then output it. The first one we will try is `sort`.

list of all the executable programs in /bin and /usr/bin, put them in sorted order, and view the resulting list.

Because we specified two directories (/bin and /usr/bin), the output of ls
would have consisted of two sorted lists, one for each directory. By including sort in our pipeline, we changed the data to produce a single, sorted list.

In [ ]:
!ls /bin /usr/bin | sort | less

### The difference between > and |

Simply put, the redirection operator connects a command with a file, while the pipeline operator connects the output of one command with the input of a second command.

```
command1 > file1
command1 | command2
```

#### Uniq: Report or Omit Repeated Lines

The uniq command is often used in conjunction with sort. uniq accepts a 
sorted list of data from either standard input or a single filename argument (see the uniq man page for details) and, by default, removes any duplicates from the list. 

In [2]:
!ls /bin /usr/bin | sort | uniq > list.txt

If we want to see the list of duplicates instead, we add the -d option

In [4]:
!ls /bin /usr/bin | sort | uniq -d > list.txt

#### wc: Print Line, Word, and Byte Counts

The wc (word count) command is used to display the number of lines, words, and bytes contained in files.

In [5]:
!wc list.txt

 1278  1278 13070 list.txt


if executed without  command line arguments, wc accepts standard input. The -l option limits its output to report only lines. Adding it to a pipeline is a handy way to count things. 

In [6]:
!ls /bin /usr/bin | sort | uniq | wc -l

1281


#### grep: Print Lines Matching a Pattern
grep is a powerful program used to find text patterns within files.

```
grep pattern filename
```

options
- i :  causes grep to ignore case when performing the search
- v :  tells grep to print only those lines that do not match the pattern

In [7]:
!ls /bin /usr/bin | sort | uniq | grep zip

bunzip2
bzip2
bzip2recover
funzip
gpg-zip
gunzip
gzip
preunzip
prezip
prezip-bin
unzip
unzipsfx
zipdetails
zipgrep
zipinfo


#### head/tail: Print First/Last Part of Files

By default, both commands print 10 lines of text, but this can be adjusted with the `-n` option

In [9]:
!head -n 5 list.txt

NF
VGAuthService
X
X11
Xephyr


In [10]:
!tail -n 5 list.txt

zipgrep
zipinfo
zless
zmore
znew


In [11]:
!ls /usr/bin | tail -n 5

zipgrep
zipinfo
zless
zmore
znew


tail has an option that allows you to view files in real time

In [12]:
!tail -f /var/log/messages

tail: cannot open '/var/log/messages' for reading: No such file or directory
tail: no files remaining


#### tee: Read from Stdin and Output to Stdout and Files

The tee program reads standard input and copies it to both standard output (allowing the data to continue down the pipeline) and to one or more files. This is useful for capturing a pipeline’s contents at an intermediate stage of processing. 

capture the entire directory listing to the file ls.txt before grep filters the pipeline’s contents

In [13]:
!ls /usr/bin | tee ls.txt | grep zip

bunzip2
bzip2
bzip2recover
funzip
gpg-zip
gunzip
gzip
preunzip
prezip
prezip-bin
unzip
unzipsfx
zipdetails
zipgrep
zipinfo


# Summing Up

There are many commands that make use of standard input and output, and almost all command line programs use standard error to display their informative messages